In [ ]:
"""
Purpose:
1. Remove extreme values based on some calculation made according to domain knowledge
2. Construct a table suggesting a range of price of each project and location
"""

In [6]:
import pandas as pd
import numpy as np
import pandas_gbq
from functools import reduce
# import scipy.stats
import datetime as dt

import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/root/jupyter/SAs/batdongsan-datalake-v0-d1b3b2296e65.json'
from google.cloud import bigquery
bigquery_client = bigquery.Client(project='batdongsan-datalake-v0')

pd.set_option('mode.chained_assignment', None)

In [ ]:
# We can read file csv in a repo instead run a query extract below:
# x = pd.read_csv('Listing Price.csv')

In [2]:
# if we one to pull the dataset directly from the GCP bigquery
qry = """
SELECT 
  CONCAT (fl.ProductID, "|",fl.price,"|",fl.area) id, 
  fl.ProductID , pr.Title, 
  fl.ProjectId, fl.area, 
  (CASE WHEN REGEXP_CONTAINS(du.unit, "/m2") THEN fl.area*fl.price*du.convert ELSE fl.price*du.convert END) price, 
  dl.CityCode  ,dl.DistrictId , dl.WardId, 
  dc.CateID, dp.projectname,
  fl.createbyuser UserID,
  MIN(PARSE_DATE("%Y%m%d", CAST(dateid AS string))) begin_date, 
  MIN(DATE_TRUNC(PARSE_DATE("%Y%m%d", CAST(dateid AS string)), month)) month,
  FROM dwh.fact_listing_v3 fl
  JOIN `dwh.dim_location` dl ON fl.LocationId = dl.Id 
  JOIN `dwh.dim_categories` dc ON dc.CateID =fl.CateID 
  JOIN `dwh.dim_unit_price` du ON du.Id = fl.UnitPriceId 
  LEFT JOIN `dwh.dim_projects` dp ON dp.ProjectId = fl.ProjectId 
  LEFT JOIN derived.fake_listing_users fk ON fk.userid = fl.createbyuser
  JOIN `staging_bds.products` pr ON pr.ProductId = fl.ProductID
      WHERE dateid BETWEEN cast(format_date("%Y%m%d",date_add(current_date(), interval -3 month)) as int64) 
                      AND cast(format_date("%Y%m%d",current_date()) as int64)
      AND publishtime IS NOT NULL
      AND du.unit != "Thỏa thuận" 
      AND fl.area > 10 AND fl.price IS NOT NULL 
      AND dc.parentname LIKE "%bán%" 
      AND du.convert IS NOT NULL
      AND fk.userid is null
      AND fl.projectid > 0
group by 1,2,3,4,5,6,7,8,9,10,11,12
"""

job = bigquery_client.query(qry)
x = job.to_dataframe()


In [3]:
x['price_bill'] = x.price / 1000
# Get rid of really really extreme values
x2 = x[(x['price_bill'] > 0.01) & (x['price_bill'] < 1000)]

x21 = x2.loc[:,
      ['ProductID', 'begin_date','month', 'area', 'projectname', 'ProjectId', 'price_bill', "Title", 
       'CateID', 'WardId', "DistrictId", "CityCode", "UserID"]]
x21.sort_values(['ProductID', 'begin_date'], inplace=True)

# Get rid of the same listing that's renewed multiple times without any changes in content
x3 = x21.drop_duplicates(['ProductID', 'price_bill', 'area'])
x4 = x3.drop_duplicates(['Title', 'price_bill', 'area'])
x4['begin_date'] = pd.to_datetime(x4['begin_date'], yearfirst=True)
x4['month'] = pd.to_datetime(x4['month'], yearfirst=True)

# Calculate price per m2
x4['m2_price'] = x4['price_bill'] * 1000 / x4.area

df_all = x4.loc[:, ['ProductID', 'begin_date','month', 'area', 'price_bill', 'm2_price',
                    'CateID', 'projectname', 'ProjectId', 'WardId', 'DistrictId', 'CityCode']]

df_all['type'] = 10
df_all['type'][df_all.area >= 200] = 200

In [4]:
def outlier_removal_std(df, group, value, left_std=3.5, right_std=4.5, median_filter=False, median_low=1.6,
                        median_high=1.9):
    median = df.groupby(group)[value].median()
    std = df.groupby(group)[value].std().fillna(0)
    count = df.groupby(group)[value].count()
    left = pd.DataFrame(median - left_std * std) - 1
    right = pd.DataFrame(median + right_std * std) + 1

    dfs = [df, median, std, left, right, count]
    df1 = reduce(lambda left, right: pd.merge(left, right, on=group), dfs)

    xx = np.array(df.columns)
    col_names = np.append(xx, ['median', 'std', 'left', 'right', 'count'])

    df1.columns = col_names
    df1 = df1[(df1[value] >= df1['left']) & (df1[value] <= df1['right'])]
    if median_filter == True:
        df1 = df1[(df1[value] <= df1['median'] * median_high) & (df1[value] >= df1['median'] / median_low)]
    else:
        pass

    df1 = df1.drop(['left', 'right', 'median', 'std', 'count'], axis=1)
    return df1

In [7]:
# filter project-related listings only
df = df_all[df_all.ProjectId != 0]

df = df[~((df.CateID.isin(['Bán căn hộ chung cư'])) & (df.area >= 500) & (df.price_bill < 10))]

# 1st filter by project, cateid
df_project_cate = outlier_removal_std(df, group=['ProjectId', 'CateID'], value='m2_price')

# 2nd filter by project, area, type, cateid
df_project_area_type_cate = outlier_removal_std(df_project_cate, group=['ProjectId', 'area', 'type', 'CateID'],
                                                value='price_bill', left_std=3, right_std=4, median_filter=True)

# 3rd filter by project, area, cateid
df_project_area_cate = outlier_removal_std(df_project_area_type_cate, group=['ProjectId', 'area', 'CateID'],
                                           value='price_bill', left_std=3, right_std=4, median_filter=True)

# final filter by project, use std and IQR calculation
median = df_project_area_cate.groupby(['ProjectId', 'CateID'])['m2_price'].median()
q1 = df_project_area_cate.groupby(['ProjectId', 'CateID'])['m2_price'].quantile(0.2)
q3 = df_project_area_cate.groupby(['ProjectId', 'CateID'])['m2_price'].quantile(0.8)
std = df_project_area_cate.groupby(['ProjectId', 'CateID'])['m2_price'].std()
i1 = median - q1
i3 = q3 - median
count = df_project_area_cate.groupby(['ProjectId', 'CateID'])['m2_price'].count()
left1 = (q1 - 2 * std) - 1
left2 = (q1 - 3 * i1) - 1
right1 = (q3 + 2.5 * std) + 1
right2 = (q3 + 3.5 * i3) + 1
left = pd.DataFrame(pd.concat([left1, left2], axis=1).min(axis=1))
left.reset_index(inplace=True)
right = pd.DataFrame(pd.concat([right1, right2], axis=1).max(axis=1))
right.reset_index(inplace=True)

In [8]:
dfs = [df_project_area_cate, median, left, right, count]

df_project_cate = reduce(lambda left, right: pd.merge(left, right, on=['ProjectId', 'CateID']), dfs)

xx = np.array(df_project_area_cate.columns)
col_names = np.append(xx, ['median', 'left', 'right', 'count'])

df_project_cate.columns = col_names

dfp = df_project_cate[
    (df_project_cate['m2_price'] >= df_project_cate['left']) & (
                df_project_cate['m2_price'] <= df_project_cate['right']) & (
                df_project_cate['m2_price'] >= df_project_cate['median'] / 1.65) & (
        (df_project_cate['m2_price'] <= df_project_cate['median'] * 1.8))]
dfp = dfp.drop(['left', 'right', 'median', 'count'], axis=1)

# Count listing by projectid and location
dfp_ct = dfp.groupby(['ProjectId', 'WardId', 'DistrictId', 'CityCode'])[['ProductID']].count()
dfp_ct.reset_index(inplace=True)
dfp_ct = dfp_ct.sort_values(['ProjectId', 'ProductID'], ascending=False)
# Keep location info
dfp_ct1 = dfp_ct.drop_duplicates(subset=['ProjectId'])

dfp_ct1.drop('ProductID', axis=1, inplace=True)
dfp1 = dfp.drop([ 'WardId', 'DistrictId', 'CityCode'], axis=1)
dfp2 = dfp1.merge(dfp_ct1, how='left', on='ProjectId')

In [9]:
def cal(df, gr, variable):
    x0 = df.groupby(gr)[variable].quantile(0.63)
    x4 = df.groupby(gr)[variable].quantile(0.15)
    x5 = df.groupby(gr)[variable].quantile(0.97)
    x1 = df.groupby(gr)[variable].count()
    
    x = pd.concat([x0, x4, x5, x1], axis=1)
    x.columns = ['AveragePrice', 'LowestPrice', 'HighestPrice', 'Count']
    x.reset_index(inplace=True)
    return x

In [10]:
project = cal(df=dfp2, gr=['CateID','CityCode',"ProjectId"], variable=['m2_price'])

project['LocationType'] = 3
project['DistrictId']=0
project['WardId']=0

project['CreatedDate'] = dt.date.today()

In [11]:
project.head(2)

,CateID,CityCode,ProjectId,AveragePrice,LowestPrice,HighestPrice,Count,LocationType,DistrictId,WardId,CreatedDate
0,40,AG,1711,45.000000,45.000000,45.000000,1,3,0,0,2022-07-07
1,40,AG,2819,51.408333,48.430556,53.517593,2,3,0,0,2022-07-07


In [12]:
df = project[['CreatedDate','CateID', 'CityCode','DistrictId','WardId','ProjectId', 'AveragePrice', 'LowestPrice',
       'HighestPrice', 'Count', 'LocationType']]

In [14]:
df[df['ProjectId']==1740]

,CreatedDate,CateID,CityCode,DistrictId,WardId,ProjectId,AveragePrice,LowestPrice,HighestPrice,Count,LocationType
2311,2022-07-07,48,SG,0,0,1740,104.000000,104.000000,104.000000,1,3
2905,2022-07-07,163,SG,0,0,1740,83.846154,83.846154,83.846154,1,3
4669,2022-07-07,324,SG,0,0,1740,65.400939,57.142857,78.483504,572,3
6068,2022-07-07,325,SG,0,0,1740,96.153846,96.153846,96.153846,2,3
7085,2022-07-07,575,SG,0,0,1740,186.105072,164.316327,187.500000,7,3


In [ ]:
# pandas_gbq.to_gbq(
#     df, 'price.bds_listing_price', project_id='batdongsan-datalake-v0', if_exists='replace',
#     table_schema = [{'name':'CreatedDate','type': 'DATE'}]
# )